In [1]:
%matplotlib inline
%pylab inline
%load_ext autotime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns
import os
import glob
import geopandas as gpd
import geopy
import plotly_express as px
import tqdm

from graphics import Point
from functools import reduce
from datetime import date
from datetime import datetime
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm.notebook import tqdm_notebook


#from dtale.views import startup
#import dtale
#import pyplot as plt
# os and glob help concat multiple csv files into one

Populating the interactive namespace from numpy and matplotlib
time: 1.58 s (started: 2021-12-16 17:53:35 -06:00)


In [2]:
from geopy.geocoders import Photon
geolocator = Photon(user_agent="measurements")

time: 0 ns (started: 2021-12-16 17:53:37 -06:00)


In [3]:
januarylistings = pd.read_csv('../data/January2021listings.csv')
januarylistings.shape

(6112, 16)

time: 31 ms (started: 2021-12-16 17:53:37 -06:00)


In [4]:
januarylistings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6112 entries, 0 to 6111
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              6112 non-null   int64  
 1   name                            6112 non-null   object 
 2   host_id                         6112 non-null   int64  
 3   host_name                       6110 non-null   object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   6112 non-null   object 
 6   latitude                        6112 non-null   float64
 7   longitude                       6112 non-null   float64
 8   room_type                       6112 non-null   object 
 9   price                           6112 non-null   int64  
 10  minimum_nights                  6112 non-null   int64  
 11  number_of_reviews               6112 non-null   int64  
 12  last_review                     52

In [5]:
januarylistings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,6422,Nashville Charm,12172,Michele,NaN,District 6,36.17315,-86.73581,Private room,40,30,674,2020-03-03,4.72,1,236
1,25613,Room in Historic East Nashville Craftsman Home,95811,Nell,NaN,District 6,36.17826,-86.74162,Private room,75,2,510,2020-03-14,3.90,1,32
2,136015,Apart. on the Ridge near Opryland,666322,Dave And Peggy,NaN,District 15,36.19464,-86.67195,Entire home/apt,103,30,50,2020-08-04,0.43,2,1
3,137063,Furnished basement room for rent,670408,Richard,NaN,District 26,36.09212,-86.71302,Private room,50,1,0,NaN,NaN,1,365
4,258817,ButterflyRoom-private bath- open Jan 26,22296,Diana,NaN,District 12,36.16249,-86.58988,Private room,27,30,96,2019-12-14,0.87,7,364


time: 16 ms (started: 2021-12-16 17:53:37 -06:00)


In [14]:
all_filenames = [i for i in glob.glob('../data/monthlistings/*.{}'.format('csv'))]

time: 16 ms (started: 2021-12-16 19:07:46 -06:00)


In [15]:
print(all_filenames)

['../data/monthlistings\\April2021listings.csv', '../data/monthlistings\\August2021listings.csv', '../data/monthlistings\\December2020listings.csv', '../data/monthlistings\\feb2021listings.csv', '../data/monthlistings\\January2021listings.csv', '../data/monthlistings\\July2021listings.csv', '../data/monthlistings\\June2021listings.csv', '../data/monthlistings\\October2021listings.csv', '../data/monthlistings\\September2021listings.csv']
time: 0 ns (started: 2021-12-16 19:07:46 -06:00)


In [16]:
#this wont work because the equation above is picking up all of the files in my data folder
#i  had to make a separate folder which only housed the "listing" data that was needed

time: 0 ns (started: 2021-12-16 19:07:48 -06:00)


In [17]:
#data is missing march - november and december
combined_months = pd.concat([pd.read_csv(f) for f in all_filenames ])

time: 265 ms (started: 2021-12-16 19:07:53 -06:00)


In [19]:
combined_months.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55268 entries, 0 to 6239
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              55268 non-null  int64  
 1   name                            55268 non-null  object 
 2   host_id                         55268 non-null  int64  
 3   host_name                       55017 non-null  object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   55268 non-null  object 
 6   latitude                        55268 non-null  float64
 7   longitude                       55268 non-null  float64
 8   room_type                       55268 non-null  object 
 9   price                           55268 non-null  int64  
 10  minimum_nights                  55268 non-null  int64  
 11  number_of_reviews               55268 non-null  int64  
 12  last_review                     4

In [20]:
#drops columns that have only NAN values
combined_months.dropna(how='all', axis=1, inplace=True)


time: 31 ms (started: 2021-12-16 19:11:16 -06:00)


In [21]:
combined_months = combined_months.drop(['neighbourhood'], axis=1)

time: 16 ms (started: 2021-12-16 19:11:16 -06:00)


In [22]:
combined_months.head()

,id,name,host_id,host_name,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,6422,Nashville Charm,12172,Michele,36.17315,-86.73581,Private room,40,30,674,2020-03-03,4.62,1,204,NaN
1,25613,Room in Historic East Nashville Craftsman Home,95811,Nell,36.17826,-86.74162,Private room,75,2,510,2020-03-14,3.81,1,125,NaN
2,39870,Close to Vanderbilt 2,171184,Evelyn,36.12523,-86.81278,Private room,60,1,262,2020-10-25,4.68,2,152,NaN
3,258817,ButterflyRoom-private bath- open May 16,22296,Diana,36.16249,-86.58988,Private room,29,30,96,2019-12-14,0.84,7,343,NaN
4,289242,"MorningstarHouse, monthly room- open may 1",22296,Diana,36.16102,-86.59207,Private room,28,30,76,2019-01-12,0.67,7,357,NaN


time: 16 ms (started: 2021-12-16 19:11:19 -06:00)


In [23]:
combined_months.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
count,5.526800e+04,5.526800e+04,55268.000000,55268.000000,55268.000000,55268.000000,55268.000000,47985.000000,55268.000000,55268.000000,18754.000000
mean,3.258022e+07,1.245387e+08,36.161135,-86.768488,253.961949,7.026543,57.151824,3.243280,18.936455,185.616360,18.207529
std,1.387132e+07,1.174331e+08,0.041666,0.053000,601.408695,17.517649,87.396312,7.977515,35.831029,131.478457,23.066427
min,6.422000e+03,1.147000e+03,35.984220,-87.035210,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000,0.000000
25%,2.120349e+07,2.819603e+07,36.141600,-86.795560,102.000000,1.000000,4.000000,0.820000,1.000000,57.000000,1.000000
50%,3.517674e+07,8.858885e+07,36.158270,-86.775840,169.000000,2.000000,24.000000,1.960000,3.000000,181.000000,10.000000
75%,4.446125e+07,1.949574e+08,36.182760,-86.743260,286.000000,2.000000,74.000000,3.580000,17.000000,317.000000,27.000000
max,5.291577e+07,4.268888e+08,36.404770,-86.535520,25000.000000,365.000000,1080.000000,493.000000,186.000000,365.000000,235.000000


time: 78 ms (started: 2021-12-16 19:11:20 -06:00)


In [24]:
combined_months['geom'] = combined_months['latitude'].map(str) + ',' + combined_months['longitude'].map(str)
combined_months['geom'][0]

0    36.17315,-86.73581
0    36.17315,-86.73581
0    36.17315,-86.73581
0    36.17315,-86.73581
0    36.17315,-86.73581
0    36.17315,-86.73581
0    36.17315,-86.73581
0    36.17315,-86.73581
0    36.17315,-86.73581
Name: geom, dtype: object

time: 109 ms (started: 2021-12-16 19:11:21 -06:00)


In [25]:
locator = Photon(user_agent='myGeocoder', timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)

time: 0 ns (started: 2021-12-16 19:11:26 -06:00)


In [26]:
from tqdm._tqdm_notebook import tqdm_notebook

time: 0 ns (started: 2021-12-16 19:11:28 -06:00)


C:\Users\YASHAC~1\AppData\Local\Temp/ipykernel_8004/3043108627.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [27]:
tqdm_notebook.pandas()

time: 0 ns (started: 2021-12-16 19:11:30 -06:00)


In [28]:
combined_months['geom'].value_counts()

36.11303,-86.76811                      323
36.15321,-86.78339                      304
36.15489,-86.77541                      216
36.15551,-86.7757                       149
36.05636,-86.74637                       96
                                       ... 
36.1566,-86.77459                         1
36.15721,-86.77284                        1
36.15655,-86.77314                        1
36.22164,-86.8065                         1
36.05729142832006,-86.74697566924686      1
Name: geom, Length: 8696, dtype: int64

time: 16 ms (started: 2021-12-16 19:11:33 -06:00)


In [ ]:
combined_months['address'] = combined_months['geom'].progress_apply(rgeocode)
combined_months.head()

  0%|          | 0/55268 [00:00<?, ?it/s]

In [25]:
#find out why these coordinates appeared 323 times 
locator = Photon(user_agent="myGeocoder")
coordinates = '36.11303,-86.76811'
location = locator.reverse(coordinates)
location.raw

{'geometry': {'coordinates': [-86.76810894443116, 36.11302835],
  'type': 'Point'},
 'type': 'Feature',
 'properties': {'osm_id': 727968560,
  'extent': [-86.7682971, 36.1132512, -86.7679417, 36.112867],
  'country': 'United States',
  'city': 'Berry Hill',
  'countrycode': 'US',
  'postcode': '37204',
  'county': 'Davidson County',
  'type': 'house',
  'osm_type': 'W',
  'osm_key': 'tourism',
  'street': 'West Iris Drive',
  'osm_value': 'motel',
  'name': 'The Iris Motel',
  'state': 'Tennessee'}}

time: 593 ms (started: 2021-12-16 12:08:44 -06:00)


In [ ]:
# the coordinates link back to The Iris Motel
# this is a small building offering 2 separate bedrooms... it shows up on AirBnB as well as other hotel booking sites.
# there is no error that this places may have been rented for that many times

In [ ]:
# i am unable to continue processing for addresses unless i remove all nan values...

In [26]:
combined_months.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284624 entries, 0 to 34
Columns: 353 entries, id to geom
dtypes: float64(325), object(28)
memory usage: 776.8+ MB
time: 16 ms (started: 2021-12-16 12:15:15 -06:00)
